# Create demo spontaneous

author: laquitainesteeve@gmail.com

purpose: create small demo dataset, upload/download to/from Dandi Archive (1 min of 40 KHz spontaneous neuropixels)

Description:
* 1 min recording
* two layers: L5, L6
* 1.2 GB

Execution time: < 10 min

Special hardware: on CPU, does not require GPU.

# Setup 

Activate dandi virtual environment (envs/dandi.yml)

```bash
python -m ipykernel install --user --name demo --display-name "demo"
```

In [1]:
%%time 

# import python packages
import os
import numpy as np
from time import time
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface
from pynwb.file import NWBFile, Subject
from pynwb import NWBHDF5IO
import uuid
from datetime import datetime
from dateutil.tz import tzlocal
from neuroconv.tools.spikeinterface import add_recording_to_nwbfile, add_sorting_to_nwbfile
print("spikeinterface", spikeinterface.__version__)

/Users/steeve_laquitaine/Desktop/EPFL/2022_2024_bbp/spikebias/envs/demo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


spikeinterface 0.101.2
CPU times: user 3.15 s, sys: 606 ms, total: 3.76 s
Wall time: 4.71 s


## Load dandiset

In [2]:
%%time

# load dandiset (npx, evoked, 20Khz)
dandiset_id = '001250'
filepath = 'sub-001-fitted/sub-001-fitted_ecephys.nwb'

# get the file path on S3
with DandiAPIClient() as client:
    asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(filepath)
    s3_path = asset.get_content_url(follow_redirects=1, strip_query=True)
print("s3_path:", s3_path)

# get RecordingExtractor
Recording = se.NwbRecordingExtractor(file_path=s3_path, stream_mode="remfile")
Sorting = se.NwbSortingExtractor(file_path=s3_path, stream_mode="remfile")

# report
print('\n', Recording)
print('\n', Sorting)

# unit-test
assert "layers" in Recording.get_property_keys(), "RecordingExtractor should contain layer property"

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/9d6/6ed/9d66ed40-af31-43aa-b4ba-246d2206dcad

 NwbRecordingExtractor: 384 channels - 20.0kHz - 1 segments - 72,359,964 samples 
                       3,618.00s (1.00 hours) - float32 dtype - 103.51 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/9d6/6ed/9d66ed40-af31-43aa-b4ba-246d2206dcad

 NwbSortingExtractor: 1836 units - 1 segments - 20.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/9d6/6ed/9d66ed40-af31-43aa-b4ba-246d2206dcad
CPU times: user 554 ms, sys: 63.1 ms, total: 617 ms
Wall time: 5.22 s


## Make the demo dataset

In [11]:
# select layer 5, 6 (most of the activity)
selected_layers = ['L5', 'L6']
channel_ids = Recording.channel_ids
channel_ids = channel_ids[np.isin(Recording.get_property('layers'), selected_layers)]
SmallRecording = Recording.channel_slice(channel_ids=channel_ids)
print("\nRecording:", SmallRecording)

# select first 2 minutes (~500 MB)
sampling_rate = Recording.get_sampling_frequency() 
start_frame = 0
end_frame = sampling_rate * 60
SmallRecording = SmallRecording.frame_slice(start_frame=start_frame, end_frame=end_frame)
SmallSorting = Sorting.frame_slice(start_frame=start_frame, end_frame=end_frame)

print("\nRecording:", SmallRecording)
print("\nSorting:", SmallSorting)

# unit-test
# - layers
# - max spike times lower than number of frames
assert (np.unique(SmallRecording.get_property('layers'))==selected_layers).all(), "layers are not correct"

max_spike_time = max([SmallSorting.get_unit_spike_train(unit_id=unit).tolist() for unit in SmallSorting.get_unit_ids()])[0]
assert max_spike_time < end_frame, "max spike timestamp should be lower that the number of frames"

# Write [TODO]


Recording: ChannelSliceRecording: 120 channels - 20.0kHz - 1 segments - 72,359,964 samples 
                       3,618.00s (1.00 hours) - float32 dtype - 32.35 GiB

Recording: FrameSliceRecording: 120 channels - 20.0kHz - 1 segments - 1,200,000 samples 
                     60.00s (1.00 minutes) - float32 dtype - 549.32 MiB

Sorting: FrameSliceSorting: 1836 units - 1 segments - 20.0kHz


## Write

In [12]:
SmallRecording.save(folder="./demo_recording", overwrite=True)
SmallSorting.save(folder="./demo_sorting", overwrite=True)

/home/jovyan/steevelaquitaine/spikebias/envs/dandi/lib/python3.10/site-packages/spikeinterface/core/job_tools.py:103: UserWarning: `n_jobs` is not set so parallel processing is disabled! To speed up computations, it is recommended to set n_jobs either globally (with the `spikeinterface.set_global_job_kwargs()` function) or locally (with the `n_jobs` argument). Use `spikeinterface.set_global_job_kwargs?` for more information about job_kwargs.
  warnings.warn(


write_binary_recording 
n_jobs=1 - samples_per_chunk=20,000 - chunk_memory=9.16 MiB - total_memory=9.16 MiB - chunk_duration=1.00s


write_binary_recording: 100%|##########| 60/60 [01:17<00:00,  1.28s/it]


NumpyFolder: 1836 units - 1 segments - 20.0kHz

## Write demo as nwb and upload to dandi archive

In [ ]:
# download a local dandiset folder with yaml config
os.system("dandi download --download dandiset.yaml DANDI:001250")

# write demo dataset to the local dandiset folder
demo_path = "./001250/spont.nwb"

# parametrize session file
nwbfile = NWBFile(
    session_description="Biophysical simulation of neuropixels in the spontaneous regime. Background noise and gain are fitted to Marques-Smith dataset.",
    identifier=str(uuid.uuid4()),
    session_start_time=datetime.now(tzlocal()),
    experimenter="Laquitaine Steeve",
    lab="Blue Brain Project",
    institution="EPFL",
    experiment_description="Biophysical simulation of neuropixels in the spontaneous regime. Background noise and gain are fitted to Marques-Smith dataset.",
    session_id="demo-npx-bio-spont",
    related_publications="https://doi.org/10.1101/2024.12.04.626805",
    keywords=["Biophysical simulation", "dense extracellular recordings", "spike sorting"]
)

# subject metadata
nwbfile.subject = Subject(
    subject_id="demo-npx-bio-spont",
    species="Rattus norvegicus",
    age="P14D",
    sex="M",
    description="Wistar Rat",
)

# bundle Extractors into nwb
add_recording_to_nwbfile(recording=SmallRecording, nwbfile=nwbfile)
add_sorting_to_nwbfile(sorting=SmallSorting, nwbfile=nwbfile)

# write nwb file locally
with NWBHDF5IO(path=demo_path, mode="w") as io:
    io.write(nwbfile)

# check file with nwbinspector <source_folder> --config dandi

# delete heavy intermediate datasets
os.system("rm -rf demo_recording -rf demo_sorting")

# upload to dandi archive
os.system(
    f"""
    export DANDI_API_KEY='210e68743286d64e84743bd8980d5771ef82bf4d';
    cd 001250;
    dandi organize {"/home/jovyan/steevelaquitaine/spikebias/notebooks/0_demo/001250/spont.nwb"} -f dry;
    dandi organize {"/home/jovyan/steevelaquitaine/spikebias/notebooks/0_demo/001250/spont.nwb"};
    dandi upload
    """
)

# clean up
os.system("rm -rf 001250")

2025-04-05 08:05:19,023 [    INFO] Logs saved in /home/jovyan/.local/state/dandi-cli/log/2025.04.05-08.05.18Z-1289.log


PATH                 SIZE DONE    DONE% CHECKSUM STATUS    MESSAGE    
001250/dandiset.yaml                             skipped   no change  
Summary:                  0 Bytes                1 skipped 1 no change
                          <0.00%                                      


rm: cannot remove 'demo_sorting': Directory not empty


256

In [22]:
print("done")

done
